# Pre-Process global _daily_ SST using `hot_to_blOb` to extract binary features

## Steps:
1. Compute Normalised Detrended Anomaly (cf. `hot_to_blOb.py::compute_normalised_anomaly()`)
2. Identify Extreme Values (i.e. above 95th percentile)

N.B.: Exploits parallelised `Dask` operations with optimised chunking using `flox` \
N.N.B.: This example using 40 years of Daily outputs at 0.25° resolution takes ~5 minutes on 128 cores

In [1]:
import xarray as xr
import dask
import intake
from getpass import getuser
from pathlib import Path

import spot_the_blOb.hot_to_blOb as hot
import spot_the_blOb.helper as hpc

In [3]:
# Start Dask Cluster
client = hpc.StartLocalCluster(n_workers=32, n_threads=4)

In [4]:
# Import 40 years of Daily EERIE ICON data

cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
expid = 'eerie-control-1950'
version = 'v20231106'
model = 'icon-esm-er'
gridspec = 'gr025'

dat = cat['dkrz.disk.model-output'][model][expid][version]['ocean'][gridspec]

In [5]:
# Load the data directly into optimal chunks

da_predictor = dat['2d_daily_mean'](chunks={}).to_dask().to.isel(depth=0).drop('depth') # Test da
time_chunk = hot.rechunk_for_cohorts(da_predictor).chunks[0]

sst = dat['2d_daily_mean'](chunks={'time':time_chunk}).to_dask().to.isel(depth=0).drop('depth')

In [10]:
# Process Data using `hot_to_blOb` helper functions:

extreme_events_ds = hot.preprocess_data(sst, threshold=95)
extreme_events_ds

In [ ]:
# Save data to `zarr` for more efficient parallel I/O

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'extreme_events_binary.zarr'
extreme_events_ds.to_zarr(file_name, mode='w')